### Performing PCA on vectors with NaNs
This notebook demonstrates the use of numpy arrays as the content of RDDs

The reason that we use numpy arrays instead of dataframes is that numpy is better in handling `nan` etries.

In numpy `5+nan=5` while in dataframes `5+nan=nan`

Suppose that the data vectors are the column vectors denoted $x$ then the covariance matrix is defined to be
$$
E(x x^T)-E(x)E(x)^T
$$

Where $x x^T$ is the **outer product** of $x$ with itself.

If the data that we have is $x_1,x_2,x_n$ then the estimates we use are:
$$
\hat{E}(x x^T) = \frac{1}{n} \sum_{i=1}^n x_i x_i^T,\;\;\;\;\;
\hat{E}(x) = \frac{1}{n} \sum_{i=1}^n x_i
$$

As it happens, we often get vectors $x$ in which some, but not all, of the entries are `nan`. In such cases we sum the elements that are defined and keep a seperate counter for each entry.

In [1]:
import numpy as np

def outerProduct(X):
  """Computer outer product and indicate which locations in matrix are undefined"""
  O=np.outer(X,X)
  N=1-np.isnan(O)
  return (O,N)
def sumWithNan(M1,M2):
  """Add two pairs of matrix,count"""
  (X1,N1)=M1
  (X2,N2)=M2
  N=N1+N2
  X=np.nansum(np.dstack((X1,X2)),axis=2)
  return (X,N)
  

In [3]:
# computeCov recieves as input an RDD of np arrays, all of the same length, and computes the covariance matrix for that set of vectors
def computeCov(RDDin):
  RDD=RDDin.map(lambda v:np.insert(v,0,1)) # insert a 1 at the beginning of each vector so that the same 
                                           #calculation also yields the mean vector
  OuterRDD=RDD.map(outerProduct)   # separating the map and the reduce does not matter because of Spark uses lazy execution.
  (S,N)=OuterRDD.reduce(sumWithNan)
  # Unpack result and compute the covariance matrix
  #print 'RDD=',RDD.collect()
  print 'shape of S=',S.shape,'shape of N=',N.shape
  #print 'S=',S
  #print 'N=',N
  E=S[0,1:]
  NE=N[0,1:]
  print 'shape of E=',E.shape,'shape of NE=',NE.shape
  Mean=E/NE
  O=S[1:,1:]
  NO=N[1:,1:]
  Cov=O/NO - np.outer(Mean,Mean)
  return Cov,Mean

#### Demonstration on a small example

In [5]:
A=np.array([1,2,3,4,np.nan,5,6])
B=np.array([2,np.nan,1,1,1,1,1])
np.nansum(np.dstack((A,B)),axis=2)

In [6]:
RDD=sc.parallelize([A,B])

In [7]:
computeCov(RDD)

#### Demonstration on real data
The following cells demonstrate the use of the code we wrote on the maximal-dayly temperature records for the state of california.

In [9]:
TMAX = sqlContext.sql("select * from weather where measurement = 'TMAX'")

In [10]:
print type(TMAX)
print TMAX.count()

In [11]:
%run /Users/yfreund@ucsd.edu/Vault

In [12]:
AWS_BUCKET_NAME = "mas-dse-public" 
MOUNT_NAME = "NCDC-weather"
dbutils.fs.unmount("/mnt/%s" % MOUNT_NAME)
output_code=dbutils.fs.mount("s3n://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
print 'Mount output status=',output_code
file_list=dbutils.fs.ls('/mnt/%s/Weather'%MOUNT_NAME)
file_list

In [13]:
US_Weather_parquet='/mnt/NCDC-weather/Weather/US_Weather.parquet/'
df = sqlContext.sql("SELECT * FROM  parquet.`%s`  where measurement=='TMAX'"%US_Weather_parquet)
df.show(5)

In [14]:
# We transform the dataframe into an RDD of numpy arrays
# remove the entries that do not correspond to temperature and devide by 10 so that the result is in centigrates.
rdd=df.map(lambda v:np.array(v[3:-4])/10).cache()
rdd.count()

In [15]:
rows=rdd.take(5)
len(rows[1])

In [16]:
import matplotlib.pyplot as plt
UnDef=rdd.map(lambda row:sum(np.isnan(row))).collect()
fig, ax = plt.subplots()
ax.hist(UnDef,bins=36)
display(fig)

In [17]:
# Remove entries that have 10 or more nan's
rdd=rdd.filter(lambda row:sum(np.isnan(row))<1)
rdd.count()

In [18]:
rdd.first()

## compute covariance

In [20]:
OUT=computeCov(rdd)
OUT

In [21]:
[(i,OUT[i].shape) for i in range(len(OUT))]

In [22]:
(Cov,Mean)=OUT

In [23]:
Cov[:10,:10]

In [24]:
from numpy import linalg as LA
w,v=LA.eig(Cov)

In [25]:
from datetime import date
from numpy import shape
import matplotlib.pyplot as plt
import pylab as py
from pylab import ylabel,grid,title

dates=[date.fromordinal(i) for i in range(1,366)]
def YearlyPlots(T,ttl='',size=(10,7)):
    fig=plt.figure(1,figsize=size,dpi=300)
    fig, ax = plt.subplots(1)
    if shape(T)[0] != 365:
        raise ValueError("First dimension of T should be 365. Shape(T)="+str(shape(T)))
    ax.plot(dates,T);
    # rotate and align the tick labels so they look better
    fig.autofmt_xdate()
    ylabel('temperature')
    grid()
    title(ttl)
    return fig
fig=YearlyPlots(v[:,:4],'Eigen-Vectors')
display(fig)

In [26]:
fig=YearlyPlots(Mean,'Mean')
display(fig)

In [27]:
Var=np.cumsum(w)
Var=Var/Var[-1]
fig, ax = plt.subplots()
#ax.plot(x, Mean, 'r')
ax.plot(x,Var)
ax.grid()
display(fig)